In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets.utils import download_url
import os
import tarfile
import datetime
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

Using downloaded and verified file: ./cifar10.tgz


In [ ]:
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

In [ ]:
data_dir = './data/cifar10'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'train']
['frog', 'truck', 'dog', 'ship', 'deer', 'airplane', 'bird', 'cat', 'horse', 'automobile']


In [ ]:
airplane_files = os.listdir(data_dir + "/train/airplane")
print('No. of training examples for airplanes:', len(airplane_files))
print(airplane_files[:5])

No. of training examples for airplanes: 5000
['1001.png', '3558.png', '0591.png', '1855.png', '4396.png']


In [ ]:
ship_test_files = os.listdir(data_dir + "/test/ship")
print("No. of test examples for ship:", len(ship_test_files))
print(ship_test_files[:5])

No. of test examples for ship: 1000
['0591.png', '0761.png', '0939.png', '0410.png', '0110.png']


In [ ]:
cifar10 = ImageFolder(data_dir+'/train', transform=ToTensor())

In [ ]:
def show_example(img, label):
    print('Label: ', cifar10.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))


In [ ]:
len(cifar10)

50000

In [ ]:
img, label = cifar10[99]

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)

In [ ]:
val_size = 5000
train_size = len(cifar10) - val_size
train_ds,val_ds = random_split(cifar10, [train_size, val_size])
len(train_ds), len(val_ds)

(45000, 5000)

In [ ]:
batch_size = 128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class ImageClassificationBase(nn.Module) :
  def training_step(self, batch) : 
    images, labels = batch
    out = self(images)
    loss = F.cross_entropy(out, labels)
    return loss
  
  def validation_step(self, batch) :
    images, labels = batch
    out = self(images)
    loss = F.cross_entropy(out, labels)
    acc = accuracy(out, labels)
    return {'val_loss' : loss.detach(), 'val_acc' : acc}

  def validation_epoch_end(self, outputs) :
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()
    return {'val_loss' : epoch_loss.item(), 'val_acc' : epoch_acc.item()}

  def epoch_end(self, epoch, result) :
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def accuracy(outputs, labels) :
  _ , preds = torch.max(outputs, dim = 1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class Cifar10CnnModel(ImageClassificationBase): 
  def __init__(self) :
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(3,32, kernel_size = 3 , padding = 1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(128, 256, kernel_size = 3 , stride = 1, padding = 1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Flatten(),
        nn.Linear(256*4*4, 1024),
        nn.ReLU(),
        nn.Linear(1024,512),
        nn.ReLU(),
        nn.Linear(512, 10))
    
  def forward(self, xb):
    return self.network(xb)

In [ ]:
model  = Cifar10CnnModel()
model

Cifar10CnnModel(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=4096, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_feat

In [ ]:
torch.cuda.is_available()

True

In [ ]:
def get_default_device() :
  if torch.cuda.is_available() :
    return torch.device('cuda')
  else :
    return torch.device('cpu')
def to_device(data, device) :
  if isinstance(data, (list,tuple)) :
    return [to_device(x,device) for x in data]
  return data.to(device, non_blocking = True)

class DeviceDataLoader() :
  def __init__(self, dl, device) :
    self.dl = dl
    self.device = device
  
  def __iter__(self) :
    for b in self.dl :
      yield to_device(b, self.device)

  def __len__(self) :
    return len(self.dl)

In [ ]:
device = get_default_device()
device

device(type='cuda')

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model = to_device(Cifar10CnnModel(), device)

In [ ]:
num_epochs = 30
opt_func = optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, model , train_dl, val_dl, opt_func)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [0], train_loss: 1.9173, val_loss: 1.6427, val_acc: 0.3873
Epoch [1], train_loss: 1.5302, val_loss: 1.4780, val_acc: 0.4518
Epoch [2], train_loss: 1.3986, val_loss: 1.3508, val_acc: 0.5064
Epoch [3], train_loss: 1.2955, val_loss: 1.2748, val_acc: 0.5473
Epoch [4], train_loss: 1.2055, val_loss: 1.1637, val_acc: 0.5749
Epoch [5], train_loss: 1.1252, val_loss: 1.0992, val_acc: 0.6106
Epoch [6], train_loss: 1.0617, val_loss: 1.0813, val_acc: 0.6126
Epoch [7], train_loss: 0.9860, val_loss: 1.0247, val_acc: 0.6303
Epoch [8], train_loss: 0.9271, val_loss: 0.9840, val_acc: 0.6539
Epoch [9], train_loss: 0.8711, val_loss: 0.9063, val_acc: 0.6861
Epoch [10], train_loss: 0.8124, val_loss: 0.8891, val_acc: 0.6921
Epoch [11], train_loss: 0.7613, val_loss: 0.8418, val_acc: 0.7122
Epoch [12], train_loss: 0.7123, val_loss: 0.8510, val_acc: 0.7028
Epoch [13], train_loss: 0.6647, val_loss: 0.8367, val_acc: 0.7127
Epoch [14], train_loss: 0.6207, val_loss: 0.7880, val_acc: 0.7346
Epoch [15], train_lo